In [ ]:
from dsc80_utils import *

In [ ]:
# Pandas Tutor setup
%reload_ext pandas_tutor
%set_pandas_tutor_options {"maxDisplayCols": 8, "nohover": True, "projectorMode": True}

# Lecture 3 – Aggregating

### Agenda

- Adding and modifying columns
- Data granularity and the `groupby` method.
- `DataFrameGroupBy` objects and aggregation.
- Other `DataFrameGroupBy` methods.
- Pivot tables using the `pivot_table` method.

You will need to code **a lot** today – make sure to pull the [course repository](https://github.com/dsc-courses/dsc80-2024-wi)

I will code a lot too, and the filled-in slides are posted online.

In [ ]:
import seaborn as sns
penguins = sns.load_dataset('penguins').dropna()
penguins

## Other `DataFrameGroupBy` methods

### Split-apply-combine, revisited

When we introduced the split-apply-combine pattern, the "apply" step involved **aggregation** – our final DataFrame had one row for each group.

<center><img src="imgs/image_0.png" width=40%></center>

Instead of aggregating during the apply step, we could instead perform a:

- **Transformation**, in which we perform operations to every value within each group.

- **Filtration**, in which we keep only the groups that satisfy some condition.

### Transformations

Suppose we want to convert the `'body_mass_g'` column to to z-scores (i.e. standard units):

$$z(x_i) = \frac{x_i - \text{mean of } x}{\text{SD of } x}$$

In [ ]:
def z_score(x):
    return (x - x.mean()) / x.std(ddof=0)

In [ ]:
z_score(penguins['body_mass_g'])

### Transformations within groups

- Now, what if we wanted the z-score within each group?

- To do so, we can use the `transform` method on a `DataFrameGroupBy` object. The `transform` method takes in a function, which itself takes in a Series and returns a new Series.

- A transformation produces a DataFrame or Series of the same size – it is **not** an aggregation!

In [ ]:
z_mass = (penguins
          .groupby('species')
          ['body_mass_g']
          .transform(z_score))
z_mass

In [ ]:
penguins.assign(z_mass=z_mass)

In [ ]:
display_df(penguins.assign(z_mass=z_mass), rows=8)

Note that above, penguin 340 has a larger `'body_mass_g'` than penguin 0, but a lower `'z_mass'`.
- Penguin 0 has an above average `'body_mass_g'` among `'Adelie'` penguins.
- Penguin 340 has a below average `'body_mass_g'` among `'Gentoo'` penguins. Remember from earlier that the average `'body_mass_g'` of `'Gentoo'` penguins is much higher than for other species.

In [ ]:
penguins.groupby('species')['body_mass_g'].mean()

### Filtering groups

- To keep only the groups that satisfy a particular condition, use the `filter` method on a `DataFrameGroupBy` object.

- The `filter` method takes in a function, which itself takes in a DataFrame/Series and return a single Boolean. The result is a new DataFrame/Series with only the groups for which the filter function returned `True`.

For example, suppose we want only the `'species'` whose average `'bill_length_mm'` is above 39.

In [ ]:
(penguins
 .groupby('species')
 .filter(lambda df: df['bill_length_mm'].mean() > 39)
)

No more `'Adelie'`s!

Or, as another example, suppose we only want `'species'` with at least 100 penguins:

In [ ]:
(penguins
 .groupby('species')
 .filter(lambda df: df.shape[0] > 100)
)

No more `'Chinstrap'`s!

<div class="alert alert-warning">
    <h3>Question 🤔 </h3>


Answer the following questions about grouping:

- In `.agg(fn)`, what is the input to `fn`? What is the output of `fn`?
- In `.transform(fn)`, what is the input to `fn`? What is the output of `fn`?
- In `.filter(fn)`, what is the input to `fn`? What is the output of `fn`?
</div>

### Grouping with multiple columns

When we group with multiple columns, one group is created for **every unique combination** of elements in the specified columns.

In [ ]:
penguins

In [ ]:
species_and_island = (
    penguins
    .groupby(['species', 'island'])
    [['bill_length_mm', 'body_mass_g']]
    .mean()
)
species_and_island

### Grouping and indexes

- The `groupby` method creates an index based on the specified columns.
- When grouping by multiple columns, the resulting DataFrame has a `MultiIndex`.
- Advice: When working with a `MultiIndex`, use `reset_index` or set `as_index=False` in `groupby`.

In [ ]:
species_and_island

In [ ]:
species_and_island['body_mass_g']

In [ ]:
species_and_island.loc['Adelie']

In [ ]:
species_and_island.loc[('Adelie', 'Torgersen')]

In [ ]:
species_and_island.reset_index()

In [ ]:
(penguins
 .groupby(['species', 'island'], as_index=False)
 [['bill_length_mm', 'body_mass_g']]
 .mean()
)

<div class="alert alert-warning">
    <h3>Question 🤔 </h3>


Find the most popular `Male` and `Female` baby `Name` for each `Year` in `baby`. **Exclude** `Year`s where there were fewer than 1 million births recorded.
</div>

In [ ]:
baby_path = Path('data') / 'baby.csv'
baby = pd.read_csv(baby_path)
baby

In [ ]:
# Your code goes here.

## Pivot tables using the `pivot_table` method

### Pivot tables: an extension of grouping

Pivot tables are a compact way to display tables for humans to read:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Sex</th>
      <th>F</th>
      <th>M</th>
    </tr>
    <tr>
      <th>Year</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2018</th>
      <td>1698373</td>
      <td>1813377</td>
    </tr>
    <tr>
      <th>2019</th>
      <td>1675139</td>
      <td>1790682</td>
    </tr>
    <tr>
      <th>2020</th>
      <td>1612393</td>
      <td>1721588</td>
    </tr>
    <tr>
      <th>2021</th>
      <td>1635800</td>
      <td>1743913</td>
    </tr>
    <tr>
      <th>2022</th>
      <td>1628730</td>
      <td>1733166</td>
    </tr>
  </tbody>
</table>

- Notice that each value in the table is a sum over the counts, split by year and sex.
- **You can think of pivot tables as grouping using two columns, then "pivoting" one of the group labels into columns.**

### `pivot_table`

The `pivot_table` DataFrame method aggregates a DataFrame using two columns. To use it:

```py
df.pivot_table(index=index_col,
               columns=columns_col,
               values=values_col,
               aggfunc=func)
```
The resulting DataFrame will have:
- One row for every unique value in `index_col`.
- One column for every unique value in `columns_col`.
- Values determined by applying `func` on values in `values_col`.

In [ ]:
last_5_years = baby.query('Year >= 2018')
last_5_years

In [ ]:
last_5_years.pivot_table(
    index='Year',
    columns='Sex',
    values='Count',
    aggfunc='sum',
)

In [ ]:
# Look at the similarity to the snippet above!
(last_5_years
 .groupby(['Year', 'Sex'])
 [['Count']]
 .sum()
)

### Example

Find the number of penguins per `'island'` and `'species'`.

In [ ]:
penguins

In [ ]:
penguins.pivot_table(
    index='species',
    columns='island',
    values='bill_length_mm', # Choice of column here doesn't actually matter!
    aggfunc='count',
)

Note that there is a `NaN` at the intersection of `'Biscoe'` and `'Chinstrap'`, because there were no Chinstrap penguins on Biscoe Island.

We can either use the `fillna` method afterwards or the `fill_value` argument to fill in `NaN`s.

In [ ]:
penguins.pivot_table(
    index='species',
    columns='island',
    values='bill_length_mm',
    aggfunc='count',
    fill_value=0,
)

### Granularity, revisited

Take another look at the pivot table from the previous slide. Each row of the original `penguins` DataFrame represented a single penguin, and each column represented features of the penguins.

What is the granularity of the DataFrame below?

In [ ]:
penguins.pivot_table(
    index='species',
    columns='island',
    values='bill_length_mm',
    aggfunc='count',
    fill_value=0,
)

### Reshaping

- `pivot_table` reshapes DataFrames from "long" to "wide".
- Other DataFrame reshaping methods:
    - `melt`: Un-pivots a DataFrame. Very useful in data cleaning.
    - `pivot`: Like `pivot_table`, but doesn't do aggregation.
    - `stack`: Pivots multi-level columns to multi-indices.
    - `unstack`: Pivots multi-indices to columns.
    - Google and the documentation are your friends!

## Distributions

### Joint distribution

When using `aggfunc='count'`, a pivot table describes the **joint distribution** of two categorical variables. This is also called a **contingency table**.

In [ ]:
counts = penguins.pivot_table(
    index='species',
    columns='sex',
    values='body_mass_g',
    aggfunc='count',
    fill_value=0
)
counts

We can normalize the DataFrame by dividing by the total number of penguins. The resulting numbers can be interpreted as **probabilities** that a randomly selected penguin from the dataset belongs to a given combination of species and sex.

In [ ]:
joint = counts / counts.sum().sum()
joint

### Marginal probabilities

If we sum over one of the axes, we can compute **marginal probabilities**, i.e. unconditional probabilities.

In [ ]:
joint

In [ ]:
# Recall, joint.sum(axis=0) sums across the rows,
# which computes the sum of the **columns**.
joint.sum(axis=0)

In [ ]:
joint.sum(axis=1)

For instance, the second Series tells us that a randomly selected penguin has a 0.36 chance of being of species `'Gentoo'`.

### Conditional probabilities

Using `counts`, how might we compute conditional probabilities like $$P(\text{species } = \text{"Adelie"} \mid \text{sex } = \text{"Female"})?$$

In [ ]:
counts

$$\begin{align*}
P(\text{species} = c \mid \text{sex} = x) &= \frac{\# \: (\text{species} = c \text{ and } \text{sex} = x)}{\# \: (\text{sex} = x)}
\end{align*}$$

<details>
    <summary>➡️ Click <b>here</b> to see more of a derivation.</summary>
$$\begin{align*}
P(\text{species} = c \mid \text{sex} = x) &= \frac{P(\text{species} = c \text{ and } \text{sex} = x)}{P(\text{sex = }x)} \\
&= \frac{\frac{\# \: (\text{species } = \: c \text{ and } \text{sex } = \: x)}{N}}{\frac{\# \: (\text{sex } = \: x)}{N}} \\
&= \frac{\# \: (\text{species} = c \text{ and } \text{sex} = x)}{\# \: (\text{sex} = x)}
\end{align*}$$
</details>

**Answer**: To find conditional probabilities of **`'species'` given `'sex'`**, divide by **column sums**. To find conditional probabilities of **`'sex'` given `'species'`**, divide by **row sums**.

### Conditional probabilities

To find conditional probabilities of **`'species'` given `'sex'`**, divide by **column sums**. To find conditional probabilities of **`'sex'` given `'species'`**, divide by **row sums**.

In [ ]:
counts

In [ ]:
counts.sum(axis=0)

The conditional distribution of **`'species'` given `'sex'`** is below. Note that in this new DataFrame, the `'Female'` and `'Male'` columns each sum to 1.

In [ ]:
counts / counts.sum(axis=0)

For instance, the above DataFrame tells us that the probability that a randomly selected penguin is of `'species'` `'Adelie'` **given** that they are of `'sex'` `'Female'` is 0.442424.

<div class="alert alert-warning">
    <h3>Question 🤔 </h3>


Find the conditional distribution of `'sex'` given `'species'`.  

**_Hint_**: Use `.T`.
</div>

In [ ]:
# Your code goes here.